# ID3 algorithm implementation

Algorithm:

In [3]:
import pandas as pd

In [4]:
columns = {
    "Drudzis": "fever",
    "Klepus": "cough",
    "Elpošanas problēmas": "short_of_breath",
    "Inficēts ar slimību X": "infected with X disease"
}
data = pd.read_excel("../id3_data.xlsx", index_col="id", names=["id"] + [v for v in columns.values()])
data

,fever,cough,short_of_breath,infected with X disease
id,,,,
1,No,No,No,No
2,Yes,Yes,Yes,Yes
3,Yes,Yes,No,No
4,Yes,No,Yes,Yes
5,Yes,Yes,Yes,Yes
6,No,Yes,No,No
7,Yes,No,Yes,Yes
8,Yes,No,Yes,Yes
9,No,Yes,Yes,Yes


Let's comeback to the task data later

In [5]:
data.to_csv("id3.csv", sep=";", index=None)

In [6]:
df = pd.read_csv("example.csv", sep=";")

In [7]:
df

,Outlook,Temperature,Humidity,Windy,Play
0,Sunny,Hot,High,False,No
1,Sunny,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Rainy,Mild,High,False,Yes
4,Rainy,Cool,Normal,False,Yes
5,Rainy,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Sunny,Mild,High,False,No
8,Sunny,Cool,Normal,False,Yes
9,Rainy,Mild,Normal,False,Yes


## Entropy for classes

Class entropy calculation formula:
$$
H(S) = -\sum_{i=1}^n p_i log_2 p_i = -(p_1log_2p_1 + p_2log_2p_2 + ...)
$$

Specific formula:
There are overall 14 records, where P = 9, N = 5(positive and negative samples)
$$
Entropy = \frac{-p}{p+n}log_2(\frac{p}{p + n}) - \frac{n}{p + n}log_2(\frac{n}{p + n}) \\
Entropy = \frac{-9}{14}log_2(\frac{9}{14}) - \frac{5}{14}log_2(\frac{5}{14}) = 0.940 \rightarrow H(S) = 0.94
$$

In [8]:
import math

-(9/14 * math.log2(9/14) + 5/14 * math.log2(5/14))

0.9402859586706311

In [9]:
positive_samples = df["Play"].where(df["Play"] == "Yes").count()
negative_samples = len(df) - positive_samples
positive_samples, negative_samples

(9, 5)

In [10]:
def class_entropy(df: pd.DataFrame) -> float:
    # Sample length
    l = len(df)
    # positive samples length
    p = df["Play"].where(df["Play"] == "Yes").count()
    # negative samples length
    n = l - p
    return -(p/l * math.log2(p/l) + n/l * math.log2(n/l))


In [11]:
h_s = class_entropy(df)
h_s

0.9402859586706311

## Entropy for features

Feature entropy calculation formula is the same:
$$
H(X) = -\sum_{x \in X} p(x) log_2 p(x)
$$

Now we calculate entropy for each column separately. Let's take `Outlook` for calculation.

Create a table of each possible value for the column, calculate their corresponding negative and positive samples and its entropy:

| Outlook  | p   | n   | H(X)  |
| -------- | --- | --- | ----- |
| Sunny    | 2   | 3   | 0.971 |
| Rainy    | 3   | 2   | 0.971 |
| Overcast | 4   | 0   | 0     |

Entropy calculations:

$$
H(sunny) = - (2/5 log_2(2/5) + 3/5 log_2 (3/5)) = 0.971 \\
H(rainy) = - (3/5 log_2(3/5) + 2/5 log_2 (2/5)) = 0.971 \\
H(overcast) = - (4/4 log_2(4/4) + 4/4 log_2 (4/4)) = 0
$$

Note that overcast has zero entropy, therefore this class is already solved as it has every value of its own group

In [12]:
df.sort_values(by="Outlook", ascending=False)

,Outlook,Temperature,Humidity,Windy,Play
0,Sunny,Hot,High,False,No
1,Sunny,Hot,High,True,No
7,Sunny,Mild,High,False,No
8,Sunny,Cool,Normal,False,Yes
10,Sunny,Mild,Normal,True,Yes
3,Rainy,Mild,High,False,Yes
4,Rainy,Cool,Normal,False,Yes
5,Rainy,Cool,Normal,True,No
9,Rainy,Mild,Normal,False,Yes
13,Rainy,Mild,High,True,No


After each entropy is calculated, we can calculate average entropy for the feature:

$$
I(Outlook) = \frac{P_S + N_S}{p + n} \cdot H(S) + \frac{P_R + N_R}{p + n} \cdot H(R) + \frac{P_O + N_O}{p + n} \cdot H(O) \\
I(Outlook) = \frac{2 + 3}{14} \cdot 0.971 + \frac{3 + 2}{14} \cdot 0.971 + \frac{4 + 0}{14} \cdot 0 = 0.693
$$

After that we can calculate the gain for column:

$$
Gain = H(S) - I(Attribute) \\
Gain(Outlook) = H(S) - I(Outlook) = 0.94 - 0.693 = 0.247
$$

Let's calculate it in Python

In [13]:
df["Outlook"].unique()

array(['Sunny', 'Overcast', 'Rainy'], dtype=object)

In [14]:
df[df["Outlook"] == "Sunny"]

,Outlook,Temperature,Humidity,Windy,Play
0,Sunny,Hot,High,False,No
1,Sunny,Hot,High,True,No
7,Sunny,Mild,High,False,No
8,Sunny,Cool,Normal,False,Yes
10,Sunny,Mild,Normal,True,Yes


In [15]:
from dataclasses import dataclass
import numpy as np

@dataclass
class Feature:
    name: str
    p: int
    n: int
    l: int
    entropy: float

features: list[Feature] = list()

for feature in df["Outlook"].unique():
    l = len(df[df["Outlook"] == feature])
    p = len(df[(df["Outlook"] == feature) & (df["Play"] == "Yes")])
    n = l - p
    # https://stackoverflow.com/a/52209380
    entropy = -(p/l * np.log2(p/l, where=(p!=0)) + n/l * np.log2(n/l, where=(n != 0)))

    features.append(Feature(
        name=feature,
        p=p,
        n=n,
        l=l,
        entropy=entropy
    ))

features

[Feature(name='Sunny', p=2, n=3, l=5, entropy=0.9709505944546686),
 Feature(name='Overcast', p=4, n=0, l=4, entropy=-0.0),
 Feature(name='Rainy', p=3, n=2, l=5, entropy=0.9709505944546686)]

In [16]:
def avg_feature_entropy(features: list[Feature], sample_size: int) -> float:
    i = 0
    for feature in features:
        i += (feature.p + feature.n)/sample_size * feature.entropy
    return i

i_outlook = avg_feature_entropy(features, 14)
i_outlook

0.6935361388961918

In [17]:
gain = h_s - i_outlook
gain

0.24674981977443933

In [20]:
np.equal(features[1].entropy, 0.0)

True

In [36]:
for attr in [a for a in df["Outlook"].unique() if a not in []]:
    print(attr)

Sunny
Overcast
Rainy


In [30]:
excl_list = ["a", "b", "c"]

def rec(l, acc):
    if len(l) == 0:
        return acc

    for a in l:
        acc += a
        l.pop()
        rec(l, acc)
    return acc

a = ""
rec(excl_list, a)
a

''

In [35]:
df[df["Outlook"] == "Overcast"]["Play"].unique()[0]

'Yes'

In [38]:
df[(df["Play"] == "Yes") & (df["Outlook"] == "Sunny")]

,Outlook,Temperature,Humidity,Windy,Play
8,Sunny,Cool,Normal,False,Yes
10,Sunny,Mild,Normal,True,Yes


In [42]:
len(df[df["Outlook"] == "Sunny"])

5

In [58]:
len(df.query("Outlook == 'Sunny' & Windy == False"))

3

In [53]:
df[df["Outlook"] == "Sunny"]

,Outlook,Temperature,Humidity,Windy,Play
0,Sunny,Hot,High,False,No
1,Sunny,Hot,High,True,No
7,Sunny,Mild,High,False,No
8,Sunny,Cool,Normal,False,Yes
10,Sunny,Mild,Normal,True,Yes


In [67]:
df["Outlook"].dtype == "O"

True

In [70]:
type(df["Windy"].unique()[0]) == bool

False

In [74]:

def _create_filter(visited_features) -> str:
    if len(visited_features) == 0:
        return ""

    def do_quotes(feature: str, attribute) -> str:
        typ = df[feature].dtype
        if typ == "O":
            return f"'{attribute}'"
        else:
            return f"{attribute}"

    return " & ".join(
        [
            f"{feature} == {do_quotes(feature, attr)}"
            for feature, attr in visited_features.items()
        ]
    )

_create_filter({"Outlook": "Sunny", "Windy": True})

"Outlook == 'Sunny' & Windy == True"

In [76]:
df[(df["Humidity"] == 'Normal') & (df["Outlook"] == "Sunny") & (df["Play"] == "Yes")]

,Outlook,Temperature,Humidity,Windy,Play
8,Sunny,Cool,Normal,False,Yes
10,Sunny,Mild,Normal,True,Yes


In [81]:
df.query(_create_filter({"Outlook": "Sunny", "Windy": True}))["Play"].unique()

array(['No', 'Yes'], dtype=object)

In [82]:
df[(df["Outlook"] == "Rainy") & df["Temperature"] == "Hot"]

,Outlook,Temperature,Humidity,Windy,Play


In [88]:
df = pd.read_csv("id3.csv", sep=";", index_col=None)

In [89]:
df

,fever,cough,short_of_breath,infected_with_X_disease
0,No,No,No,No
1,Yes,Yes,Yes,Yes
2,Yes,Yes,No,No
3,Yes,No,Yes,Yes
4,Yes,Yes,Yes,Yes
5,No,Yes,No,No
6,Yes,No,Yes,Yes
7,Yes,No,Yes,Yes
8,No,Yes,Yes,Yes
9,Yes,Yes,No,Yes


In [96]:
df[(df["short_of_breath"] == "No") & (df["fever"] == "Yes")]["infected_with_X_disease"].value_counts().to_dict()

{'No': 2, 'Yes': 1}